In [1]:
import pandas as pd 

import py2neo


import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
def query_database(query):
    # REMEMBER TO BE CONNECTED TO IMPERIAL WIFI!
    graph_db = py2neo.Graph("https://dsi-bitcoin.doc.ic.ac.uk:7473/db/data/", auth=("guest_ro", "imperialO_nly"))
    return graph_db.run(query)

def get_block_data(first_block, last_block):
    query_string = """
                    MATCH (b:Block) <-[:MINED_IN]- (t:Tx) <-[:IN]- (txi:TxIn) <-[:UNLOCK]- (iadr:Address)
                    WHERE b.height >= {} AND b.height <= {}
                    MATCH (txi) <-[:SPENT]- (txo_in:TxOut) 
                    MATCH (oadr:Address) <-[:LOCK]- (txo_out:TxOut) <-[:OUT]- (t)
                    
                    RETURN iadr.address as iadr, oadr.address as oadr, txo_in.value as input_val, txo_out.value as output_val, ID(txo_in) as id_txo_in, ID(txi) as id_txi, ID(t) as id_t, ID(txo_out) as id_txo_out
                    """.format(first_block, last_block)
    return query_string

result = query_database(get_block_data(400000,400000))
df = result.to_data_frame()

addresses = list(df.oadr.value_counts().index)

blk = 400000

def seen_before1(block, adr):
    params = {'block':block,'adr':adr}
    query_string = """
                    MATCH (a:Address) <-[:LOCK]- (to:TxOut) <-[:OUT]- (t1:Tx)-[:MINED_IN]->(b1:Block)
                    WHERE a.address = "{adr}" AND b1.height<{block}
                    RETURN b1.height LIMIT 1
                    """.format(**params)
    
    data_fr = query_database(query_string).to_data_frame()
    if data_fr.empty:
        return False
    return True

def seen_before2(block, adr):
    params = {'block':block,'adr':adr}
    query_string = """
                    MATCH (a:Address) -[:UNLOCK]-> (ti:TxIn) -[:IN]-> (t2:Tx)-[:MINED_IN]->(b2:Block)
                    WHERE a.address = "{adr}" AND b2.height<{block}
                    RETURN b2.height LIMIT 1
                    """.format(**params)
    
    data_fr = query_database(query_string).to_data_frame()
    if data_fr.empty:
        return False

    return True

not_seen =[]
import time

temptime = time.time()
starttime = time.time()

for adr in addresses:
    if seen_before1(blk, adr) or seen_before2(blk, adr):
        continue
    else:
        not_seen.append(adr)
    print(adr, " processed in ", time.time()-temptime)
    temptime = time.time()
    
print("Total time to process: ", time.time()-starttime)

16HzNspMpPedrNocntGxDTqk7cqsb3Q7ND  processed in  0.28920674324035645
1MzpFjuQ1UC8v9khaTxrzdt2tqqKpkRJNt  processed in  0.4400346279144287
1FYVhnRVyHjsP6SXrViTmR3oExnrhtz3tX  processed in  0.01758289337158203
188z786vHfSmjoToGLhVyNLXsBuYuKVbci  processed in  0.08354687690734863
1C8Ks1fjfDDNEYrs3hJKLERA1HDnNTfLq7  processed in  0.15293550491333008
13bKWQN84UL22ZHKQarvmzyRP7jy7DVz7x  processed in  0.15831828117370605
1NZjqaXMNcu7wir7QbBYnpC4DQA8BDFVcR  processed in  0.37097787857055664
1798F49znRhX5MJPmRYRkrB5en7DtG7SM3  processed in  0.004784107208251953
1ETSP9C7FDVm37iWs1yTR811tDrWSviHns  processed in  3.001511812210083
12oZFFh2mWxCbE8auph1uSZHzSCWixoA2i  processed in  0.01697707176208496
1C3dZ31vPHbV3G3HRecVe9KLZYUqBTAVpn  processed in  0.01613926887512207
1HX4EL1uPgaPquW8tYgQ7oRcsfZ7VerZ8b  processed in  0.016571521759033203
1Jxeewj2mCS1qBZcArfPQUR4GWWPtgf2i1  processed in  0.014203548431396484
16ytLDsm9pCimyJ3apxQ9CUbp7gJ7mPW9U  processed in  0.019427776336669922
1Q2qqboEfDJu5mDXNZ7

16cyRbxjESLsv16Udo4a9N24X3fSMFnX5x  processed in  0.01522064208984375
1BbVMNtHj1gqkKkNWRNVQJBVX4hmhBTVrg  processed in  0.019156455993652344
15XebgNS25V9jHroZudqMXiVHydUZvwCG9  processed in  0.013278484344482422
1C6gwXcmP1JUhnGFU9CXshqM8ku5YeesqL  processed in  0.01115107536315918
1DKFviXdLb1UGywHv4hP2QJdp3Cr2qkjDM  processed in  0.025471210479736328
1ExTLyHDfrcDDs1aZJmjinoJPEb4Z6KC9A  processed in  0.012234210968017578
1HV7Lyo7wQ6f9YtfLQnaf25AeRy7eiP2Tn  processed in  0.01823115348815918
1CKBW51F2mFzRXMprsdLQrT31rqEYuP4an  processed in  0.01691412925720215
1EAgzGJWm4Lv4h2AXFFNsKoxPzdUwAHiex  processed in  0.011224985122680664
1Pu1fQevBPHMpi4f6jNgiXH6z6ihuGbtYX  processed in  0.016289472579956055
1Ha17TRwp58qmsGThYqzwcDFufNz3JuiN5  processed in  0.016771554946899414
1PzqAVmdpNSwr3V8osgVw1P7RrXkoHGT76  processed in  0.011850357055664062
19TSJdsRxfQSNuTameUygyYcXJmbRWVdXL  processed in  0.013934135437011719
1JpKvrJtmoS58y62WSDQ1dFmMsi9Ho3erE  processed in  0.01762700080871582
1DcDtqnwiDX

1NrgMWyo76wrn2NKJY4z143HBJBBzeg74H  processed in  0.012912988662719727
1Fvm8BYxjxNBJZXUpEs1qdXKBsadhVA4BT  processed in  0.01514577865600586
1MMhvL3wVsRnYwFZQ5kCktDSNU2jsDk9Np  processed in  0.012843608856201172
1GAUaue56HPhoQT6FXibD2euigRB3Qb1rG  processed in  0.5666983127593994
197NdraAGmPqnN5QLxLBf1u7JA649E8q6Z  processed in  0.013602018356323242
126bh5QAkaXfWceveJYLa1LSF1AvCBPufC  processed in  0.016803503036499023
1B4mRmmXfo1Su2Hj9g5KoYaSVxf8u4mnbj  processed in  0.012888669967651367
1EPb78Dw5B1Fb6cgSGg5dwwZ4dzonakSzM  processed in  0.017607927322387695
12ufk4V1vAQNrsmMPQrmneZjcVxrmfYHV3  processed in  0.021631956100463867
1PPMCM7nbntWENoGW2pK5cfzQmbtaH6bN2  processed in  0.027547836303710938
1HkMUUbsWsvyMNj7CPKsZYR11cx1GD1DLZ  processed in  0.014045238494873047
1AtBbNhuQtD7HQjnyo35rKG9ey4Ehh4Ap9  processed in  0.016242504119873047
1KTtWKrW16iZLZD7zgsh54WrmJd5KKpcEy  processed in  0.6784555912017822
1MWhggZsaKeE1ojbt9EQvextYbZb9GYrpa  processed in  0.0216367244720459
1JZpDQZ4SWXF7

155aTUJp6hLxfbRNMnWaJMBgjyNkyfMKRa  processed in  0.06657671928405762
1PbDUmeZeJNj5npS8buPqo7bZtBSYskwbZ  processed in  0.02905416488647461
1MMpwxQ6padT1weYSL29H6eiqBjcR2D6Ej  processed in  0.0237882137298584
13jFCebPDodqaYcHH51czuJTNJvwqQdKMw  processed in  0.13627243041992188
1F4iBAaCzuvDmeK62qxUTJQydDV46pNh3a  processed in  0.020230770111083984
16wkAR4KTz34Wg1myjpL5GQPL1Kby4wSAA  processed in  0.017837047576904297
1KCMFdXQ26HxvN354ESHubxA5Wd5WnCiYm  processed in  0.1889951229095459
1FPugDyNaDz3qXq4VnjZdQzSJqjXf4vTby  processed in  0.04084014892578125
16ccLvdnW4z4g1zhfWQtauNRxUddS3XNYc  processed in  0.030812978744506836
1PhmEfQ5zPxsmKUWtiVgYaS7KAWQ6SkAFC  processed in  0.03368210792541504
166EpnXAwEUoKRppj1czEzAEe54QRqpuu9  processed in  0.022207260131835938
1KDGRKJaSjDje8EL9ZKrq391H988fDTCo  processed in  0.019176244735717773
1FvCmatoRPZbLH3ioHPxT3RJbnDUewtCWb  processed in  0.023140430450439453
16RBmP8JMQtYSGbsg6wMJ4bS4toKU6AwEy  processed in  0.020765066146850586
13gpfVYcTo1gmCKQ

1FmFGnfv9ytD5kvjWZcrYaqnPC8gYb6Kf5  processed in  0.01717066764831543
1N4iQhRCSCN1K72ku4osg9fpXMEsbMebJr  processed in  0.02055978775024414
18tszoFTH9QBkgiLQmUFnJy5ignyijXTpm  processed in  0.052870750427246094
1p3BWoRrr1iFYE5C9tkajCXVuQ3sGoy71  processed in  0.015858173370361328
1E2K6dU1esc6FNcSMAFbwPKZh6Q9DGdQHb  processed in  0.01607203483581543
1Aki8Ls8RCJEwg3g1rMQxwwwXdMP7QzqeC  processed in  0.03416919708251953
19VBcbWt27F2vF88B6s2s9LQTdFspiAqAf  processed in  0.025483369827270508
1Cpqkc2xpCwt1P8vEvrQETJSUBog9BUUie  processed in  0.01533365249633789
1AsMhtQwewBEzVnw3iEXQPD4HwAstYTw4B  processed in  0.08370733261108398
12VzchyocdjKFmyrh3WcT1BxPKnQgqetEo  processed in  0.019390344619750977
1Mq6MTYgXvAh7muSyEPtHHLXxXKXGjV9K2  processed in  0.05105924606323242
189n9c9c8XrowcZWvUozPPKdkJkviE68m7  processed in  0.02601146697998047
1EBVYG94Bi1N6K9h3GaJWYDWG1Zp9K7Na5  processed in  0.026204824447631836
18aSKpQDCgoWg7QWikQZ83xQfKuugaXeJJ  processed in  0.035714149475097656
18SjcVSVWqHePdz

...

13gyCatvwqPyRi9L2ezwH86NVnEjkKYjas  processed in  0.04433774948120117
19Uif2UJmdKCHEFBSVnopN1WENwu8ezgNY  processed in  0.2707209587097168
1K8nuaxctvKSKBdGn8HS2MTpPXQ4peZAaF  processed in  0.03835344314575195
1FVFusXt4Uhz6cHeM6uYDndctFEkXp3gFo  processed in  0.08588480949401855
16rRNP7ujugrGKdmWJL5NgN6uoJfSBpGpn  processed in  0.03983116149902344
1JuoG2dxenqF6VsTSwEH6ZsQBM9LC84ETB  processed in  0.030478954315185547
16enR52g65KHMRZrbqBKGTmEsPTwL4u17a  processed in  0.04007315635681152
152CfArzCe3F45h3tyV3jEteE63pm9iUuv  processed in  0.043538570404052734
166sepFPAFdpEowDvvgVpZGY4XRh7DxgfB  processed in  0.03477120399475098
1BHZ6yUygBjBa7MquJJVP5N2GbetR6mQxh  processed in  0.02928018569946289
1D8v6yskngtBPTzyLsJfT3nU3YA94NVxLC  processed in  0.11604642868041992
1wCRmTCmw24SxThf3MG9vEMo6eRjpyaPo  processed in  0.03684043884277344
15tJadgJsCzS7c35X4miyE5oqAeCwePy9w  processed in  0.02637505531311035
14BbGq5nYrCfAiDx3dKDr5e32rupSGgUgY  processed in  0.03505301475524902
15ebw3LHD22a6hJWXQKB

1GEjAWNHSbvykRh8fAox4SLidwNurzgMgz  processed in  0.135390043258667
134Se1NCXhQaMNx8oUtK7jB9T7Tramiqig  processed in  0.07608747482299805
165u14HLrVBwGjuWxc4HnuNGezhg5qTucu  processed in  0.03316020965576172
15FBPizL5Vgb8G4Vvfg2WUgzfkhWYyFN72  processed in  0.16515111923217773
12hjnV3PMJ31CX9TcdYrfxMfYZaGkLgAe6  processed in  0.024232149124145508
1DgVobUXgtvse1XxxFELp7ES7A9QLXeDyw  processed in  0.03006768226623535
1GqCoRYthGx9RU8pu92BXdXEozW8C9uz2w  processed in  0.02431321144104004
1K4F625rWC3VQeVP7yiKesP4dvhsdyRJYu  processed in  0.10479331016540527
1PA5qo56SDYiGoC2qk4S5fteupgY4N5DEL  processed in  0.026614665985107422
1EBeRZYviLJ3QEBCvxhPpDQquqLynvinNz  processed in  0.06787562370300293
1PAR5ZFvPByT8iGW3G4vutQ5PtjzzHxCrh  processed in  0.1091606616973877
19z2NhMBEQnPVAsASX3bvQmgbMWHP7SpbK  processed in  0.04260134696960449
1Jt2QwkojGiy3GuCcm6LDaYmtcR4m2Aj5N  processed in  0.05062055587768555
1Le1zT45Me6xSEctoBs1pMDAFTDzj7KeDk  processed in  0.5414273738861084
15LJUBdvTBy1FUss8gpeUJ

15LHMxhyB2AEryq58UoKrz4Q9Hu6WT9w7c  processed in  0.27440667152404785
1Fvj9QDESLMr7hUPS7SApR5Az8DKPd49sB  processed in  0.12308216094970703
1GRqLjKJamHawCx7ykzoeB4iiW7zAjxi3A  processed in  0.0362093448638916
1AjnGrBQytuh4wFcRLEAvyk1CsnJxiv3YE  processed in  0.04070925712585449
18PCFspxEhR6GxWeCGZEqYPXnu8eoHFZa6  processed in  0.038231849670410156
1MYb8cqynKf8XL7Tag8Bms9yptc46dTj38  processed in  0.04160451889038086
14eZEdcNAoY82jJANhn1HUYdiKDXy1gufi  processed in  0.04062366485595703
12zrDZ7miFo67SXoaodSN3UkZcnWChMiRG  processed in  0.09902787208557129
168QTLf7fMkzCRnJFAxeoZJeawaWJD6mFa  processed in  0.04125475883483887
1HXar17CNuzeZJRDZMggkoxEbQkBctQDCx  processed in  0.0486912727355957
1JZ8XbQydXKgphnJ2t8pZ2orTrXDhVzFAg  processed in  0.04377102851867676
19o3SDR46JVbmELLAAfzyLZ6hfrx9JceW4  processed in  0.2510721683502197
1QKa5LGGJDx2qAK6sE96X4QFdbv1Bq8yS2  processed in  0.046843528747558594
12MJA2BHkfzFckkK8ojPe7pyuH1rVPffQF  processed in  0.07871484756469727
1JpanaBxMpqHrNSMorJqB

19sMCQZKaqmwZVruWdTyYgNKLcP1MsFhxp  processed in  0.060776710510253906
1GHeeH13Gcqjco91YpM9G3DW3qpFRJohX7  processed in  0.15159225463867188
1DVHzqPidQvnTzCVe9fAmSU2iQaYaAKZnz  processed in  0.15961885452270508
1JuEeADC6BCqjR2DeEZMsMHA1yp8yhqDkT  processed in  0.05237746238708496
194XYNaVGZsnHXB6iJHEWuXYhFQJggNEuY  processed in  0.03839898109436035
1Q5P3ajZCBtZS19by7RJpt6Zfv6rZdvmjk  processed in  0.046122074127197266
1Lyi6DgLCofX3QsJcQ43uaZv9eG8m6xNbd  processed in  0.035532236099243164
1M85oDq1N9nqwrnDSKFMXtfpw853orBYC4  processed in  0.10222125053405762
1ANjczfDxN2F9uWsokKn37HHDjRzBosSN7  processed in  0.06202888488769531
1HfVtT2ku57UGzrDF1JNnfphxRTm38f2ry  processed in  0.04705381393432617
1JHHNfREd1V2xbWRiRTjfMJWMiSdiQ4Pf8  processed in  0.02858567237854004
1KT9e9aUPi6yZzUFck2pfZrtenPt5HWf8b  processed in  0.0655829906463623
1MLJUxvwJXGcciKgQvicMNTxNEm5gDBr6h  processed in  0.031945228576660156
12QWah6VhDv69eU5qiKW9nbUqSxzoHAz4v  processed in  0.032593488693237305
1Bemb3h9sWvYrJiQ

12BKEGkzA7iCwVJnfQbZFW7Vpqd6uDAUPo  processed in  0.037980079650878906
15jsF7dR4Xudcdr9yGFsnPUWguPs3DUzTQ  processed in  0.09181571006774902
1FiSTrsfrt2nHqxjRzi8f5v6q14wRzRVXU  processed in  0.03660702705383301
1N5eVZE4gGsGL8m27uHwqCecmMwrS5M35v  processed in  0.03710651397705078
1LxjmnFMZmERj7MuVznztFiEM8gvqeWxVE  processed in  0.032727956771850586
1MQSFAsjaYjBsEimvJz9Fusrm7thShpKSX  processed in  0.030541419982910156
1B93XtnKsudzXauTQZidxZPdLZQ3tyjuNa  processed in  0.04348349571228027
1C37TBmCY2udDKEyivYRcntH3CkFpax9SF  processed in  0.036512136459350586
1EJR9joMZeQzH3BQtynfGpXotpejRbjYN6  processed in  0.0663449764251709
1JHPYbAWdAQm1gvHDjVjQbF3RqYBQB5ZAZ  processed in  0.034857749938964844
1KeWCJFaCDAcSBnxkhKQffdhiPnBBhmY4H  processed in  0.10603785514831543
15fdrb6wvMuXTuu42G2rs8G5ex2H8FXxGS  processed in  0.04366350173950195
1B9LCL5ixNUFkterA84ZXHgf6D2Fmaupgc  processed in  0.09467053413391113
1AhZtYkru6PqJ6WPCHfW1jnvUEMdk8Le9U  processed in  0.03491616249084473
1Dqbhy2ZGi1UToFy

In [6]:
#Write to file

import pickle
with open('outfile','wb') as f:
    pickle.dump(not_seen,f)

In [ ]:
addresses1 = list(df.oadr.value_counts()[df.oadr.value_counts()>1].index)
addresses2 = list(df.oadr.value_counts()[df.oadr.value_counts()==1].index)

In [ ]:
blk = 400000
not_seen = [adr for adr in addresses1[:5] if not seen_before1(blk, adr) and not seen_before2(blk, adr)]